In [2]:
import psycopg2
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
class trip:
    def __init__(self, init_pos, end_pos, init_time, end_time):
        self.taxi_init_pos = init_pos
        self.taxi_end_pos = end_pos
        self.taxi_init_time = init_time
        self.taxi_end_time = end_time

    def bus_init(self, init_pos, init_stop, init_time, bus_id, trip_id):

        self.bus_init_pos = init_pos
        self.bus_init_stop = init_stop
        self.bus_init_time = init_time

        self.bus_id = bus_id
        self.trip_id = trip_id

    def bud_end(self, end_pos, end_stop, end_time):
        self.bus_end_stop = end_pos
        self.bus_end_pos = end_stop
        self.bus_end_time = end_time
        



In [2]:
# get taxi initial points
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

initial_points = []
sql = '''
select st_transform(initial_point, 3763)
from taxi_services 
'''
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

for row in results:
    initial_points.append(row[0])
print(initial_points[:5])

['0101000020B30E0000DCCDA3092D31E3C09F8BE867904B0441', '0101000020B30E00002877A4E9642FE2C0445BEC854A020441', '0101000020B30E00007D59F7E82972E3C0B1967DA256090441', '0101000020B30E000074BC0AE11660E4C07BDDB2DBD7290441', '0101000020B30E00007C232BBD37B0E3C025F7A7F230270441']


In [3]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# get the closest bus stop for each initial point
initial_stops = []
for initial_point in initial_points[:5]:
    sql = f'''
    select 
        stop_id,
        st_transform(t.initial_point, 3763) as ip,
        st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763),
        st_transform(t.final_point, 3763),
        t.initial_ts
    from taxi_services t
    cross join stops s
    --where st_transform(t.initial_point, 3763) = '{initial_point}'
    order by st_distance(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763), '{initial_point}')
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    for row in results:
        initial_stops.append((row[0], row[2], row[3], row[4]))
print(initial_stops[:5])

[('COMB2', '0101000020B30E00008683A8946A29E3C03192DC5FB84B0441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('COMB1', '0101000020B30E0000F656F70DDF25E3C050971C19D54B0441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('PSOC1', '0101000020B30E0000F18341A7953AE3C0DDEE33D0A1460441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('LIMA1', '0101000020B30E0000F89AD339DA1BE3C023B4EA3CE8470441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621), ('LIMA2', '0101000020B30E0000B703E7B0161FE3C0EF8E9D1D77460441', '0101000020B30E0000822D6193E0C3E3C0106715740C000441', 1420070621)]


In [4]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

stop_ids = []

# todos os autocarros que param nessa initial_stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    select trip_id from stop_times where stop_id = '{initial_stop[0]}'
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    stop_ids.append(results[0])
print(stop_ids[:5])

[('5M_1_D_1',), ('5M_0_D_8',), ('701_0_D_11',), ('806_0_D_15',), ('806_1_D_6',)]


In [20]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# now we get the final stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    with t2 as ( SELECT DISTINCT ON (t.route_id) t.route_id, st.trip_id, st.departure_time, '{datetime.fromtimestamp(initial_stop[3]).time()}', st.stop_id
                    FROM stop_times st
                    JOIN trips t ON st.trip_id = t.trip_id
                    WHERE st.stop_id = '{initial_stop[0]}'
                      AND st.departure_time > '{datetime.fromtimestamp(initial_stop[3]).time()}'
                    ORDER BY t.route_id, extract(epoch from (st.departure_time - '{datetime.fromtimestamp(initial_stop[3]).time()}')))

    select st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}') as p,
    t2.route_id, 
    t2.trip_id,
    s.stop_id
    from t2 
    join stops s on s.stop_id = t2.stop_id
    group by t2.route_id, t2.trip_id, s.stop_id
    order by st_distance(st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}'), '{initial_stop[2]}')
    limit 1
    '''

    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    print(results, "\n")

[('0101000020B30E00008683A8946A29E3C03192DC5FB84B0441', '5M', '5M_1_D_1', 'COMB2')] 

[('0101000020B30E0000F656F70DDF25E3C050971C19D54B0441', '5M', '5M_0_D_2', 'COMB1')] 

[('0101000020B30E0000F18341A7953AE3C0DDEE33D0A1460441', '5M', '5M_0_D_2', 'PSOC1')] 

[('0101000020B30E0000F89AD339DA1BE3C023B4EA3CE8470441', '806', '806_0_U_19', 'LIMA1')] 

[('0101000020B30E0000B703E7B0161FE3C0EF8E9D1D77460441', '806', '806_1_S_1', 'LIMA2')] 

[('0101000020B30E00004CE3657D4633E3C0B225F9E870520441', '204', '204_1_U_113', 'ACST2')] 

[('0101000020B30E000034A182BD2D4AE3C018380C77814E0441', '204', '204_1_U_113', 'BJ1')] 

[('0101000020B30E0000D369B4374A50E3C039BC3A56E64E0441', '204', '204_0_D_1', 'BJ3')] 

[('0101000020B30E0000612ADAAABA0DE3C0554D5051404E0441', '806', '806_0_U_19', 'COMB3')] 

[('0101000020B30E00009B4D46850F48E3C0D29E9A020D440441', '5M', '5M_1_D_1', 'PSOC2')] 

[('0101000020B30E00004A94150D220DE3C0BF224527D44E0441', '806', '806_1_S_1', 'COMB4')] 

[('0101000020B30E000032F4068C3643E3C08

In [22]:
import psycopg2

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()


# complete query but slow
sql = '''
select 
    t.id,
    s1.stop_id,
    s2.stop_id,
    st_astext(st_transform(t.initial_point, 3763)) as initial_point,
    st_astext(st_transform(t.final_point, 3763)) as final_point,
    st_astext(st_transform(st_setsrid(st_point(s1.stop_lon, s1.stop_lat), 4326), 3763)) as stop_point_init,
    st_astext(st_transform(st_setsrid(st_point(s2.stop_lon, s2.stop_lat), 4326), 3763)) as stop_point_end,
    t.initial_ts,
    t.final_ts,
    s2.departure_time, 
    --no departure time for last stop yet (need a new join this stop_times and that is slow :( )
    s2.trip_id --to later find the final deprature time
from taxi_services t
JOIN LATERAL ( --get init stop_id, s1.stop_id
    SELECT 
        stop_id,
        stop_lon,
        stop_lat
    FROM 
        stops s
    ORDER BY 
        ST_Distance(
            ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763), 
            ST_Transform(t.initial_point, 3763)
        )
    LIMIT 1
) AS s1 ON TRUE
JOIN LATERAL ( --get final stop_id, s2.stop_id
    WITH t2 AS ( --get all trip_ids that pass the init point, only the first trip after intiial_ts from each route will be selected
        SELECT DISTINCT ON (ts.route_id) 
            st.departure_time,
            st.trip_id,
            st.stop_id
        FROM 
            stop_times st
        JOIN 
            trips ts ON st.trip_id = ts.trip_id 
        WHERE 
            st.stop_id = s1.stop_id
            AND 
            st.departure_time > CAST(to_timestamp(t.initial_ts) AS TIME)
        ORDER BY 
            ts.route_id, 
            EXTRACT(EPOCH FROM (st.departure_time - CAST(to_timestamp(t.initial_ts) AS TIME)))
    )
    SELECT --get all point stops that pass on the route
        st.departure_time,
        t2.trip_id,
        --t2.stop_id as init_stop_id,
        st.stop_id,
        s.stop_lat,
        s.stop_lon
    FROM 
        t2 
    JOIN 
        stop_times st ON st.trip_id = t2.trip_id
    JOIN 
        stops s ON s.stop_id = st.stop_id
    GROUP by 
        st.departure_time,
        t2.trip_id, 
        --t2.stop_id,        
        st.stop_id,        
        s.stop_lat,
        s.stop_lon
    ORDER BY 
        ST_Distance(
            ST_ClosestPoint(
                ST_Collect(ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763)),
                ST_Transform(t.final_point, 3763)
            ), 
            ST_Transform(t.final_point, 3763)
        )
    LIMIT 1
) AS s2 ON TRUE
LIMIT 10000;
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
print(len(results), "\n")
print(results)

10000 

[(7269, 'COMB2', 'CORD2', 'POINT(-39305.4074267407 166258.050736514)', 'POINT(-40479.0179906739 163841.556681446)', 'POINT(-39243.3306467599 166263.046807424)', 'POINT(-40574.6683797783 164096.948888512)', 1420070621, 1420071792, datetime.time(6, 44), '703_1_B_1'), (7270, 'CMS4', 'CORD7', 'POINT(-41608.4657867441 165864.987554511)', 'POINT(-40491.5747998201 164166.478588727)', 'POINT(-41678.662140863 165888.177417517)', 'POINT(-40541.128929977 164118.299693258)', 1420070905, 1420071378, datetime.time(6, 27, 13), '208_1_V_1'), (7271, 'BSS3', 'BS10', 'POINT(-42683.8975228505 165780.231992565)', 'POINT(-40273.5194220084 165070.862992153)', 'POINT(-42793.8908064418 165807.262484494)', 'POINT(-41592.0753122211 165327.793695076)', 1420070708, 1420071057, datetime.time(6, 0), '503_0_U_45'), (7272, 'AANT5', 'CRJ2', 'POINT(-37772.6753019521 166280.950647465)', 'POINT(-40040.3571925886 163640.231674948)', 'POINT(-37792.1517335032 166100.028290137)', 'POINT(-37576.1084555906 165679.025513

In [27]:
#build a dictionary of the type dic[trip_id][stop_id] = departure_time
sql = ''' 
    SELECT trip_id, stop_id, departure_time 
    FROM stop_times;
'''

stop_times = {}

cursor_psql.execute(sql)
results2 = cursor_psql.fetchall()
#print(len(results2), "\n")
#print(results2)

for r in results2:
    if stop_times.get(r[0]) == None:
        stop_times[r[0]] = {}
    stop_times[r[0]][r[1]] = r[2]

print(stop_times)

{'508_1_Y_23': {'CPOV2': datetime.time(20, 23, 43), 'ABX2': datetime.time(20, 24, 22), 'BRG2': datetime.time(20, 25, 13), 'MSB2': datetime.time(20, 26, 29), 'TPG2': datetime.time(20, 27, 34), 'VDTM2': datetime.time(20, 28, 17), 'TLN2': datetime.time(20, 29, 16), 'ESP2': datetime.time(20, 30), 'GNDV2': datetime.time(20, 30, 39), 'TPI2': datetime.time(20, 31, 26), 'AJA2': datetime.time(20, 32, 2), 'PCRQ2': datetime.time(20, 32, 43), 'CAL2': datetime.time(20, 33, 16), 'CUS4': datetime.time(20, 34, 33), 'TFV2': datetime.time(20, 35, 40), 'AVLH2': datetime.time(20, 36, 38), 'RDS2': datetime.time(20, 37, 35), 'PLG2': datetime.time(20, 38, 38), 'CSXN2': datetime.time(20, 39, 21), 'FGT2': datetime.time(20, 40, 10), 'CSXS2': datetime.time(20, 41, 20), 'MTB4': datetime.time(20, 42), 'SRPT2': datetime.time(20, 43, 19), 'PPR2': datetime.time(20, 44, 13), 'CI22': datetime.time(20, 45, 20), 'HPRL2': datetime.time(20, 46, 1), 'CVI2': datetime.time(20, 47, 11), 'QTAM2': datetime.time(20, 48, 34), 'CZV

In [23]:
print(results[:100])

[(7269, 'COMB2', 'CORD2', 'POINT(-39305.4074267407 166258.050736514)', 'POINT(-40479.0179906739 163841.556681446)', 'POINT(-39243.3306467599 166263.046807424)', 'POINT(-40574.6683797783 164096.948888512)', 1420070621, 1420071792, datetime.time(6, 44), '703_1_B_1'), (7270, 'CMS4', 'CORD7', 'POINT(-41608.4657867441 165864.987554511)', 'POINT(-40491.5747998201 164166.478588727)', 'POINT(-41678.662140863 165888.177417517)', 'POINT(-40541.128929977 164118.299693258)', 1420070905, 1420071378, datetime.time(6, 27, 13), '208_1_V_1'), (7271, 'BSS3', 'BS10', 'POINT(-42683.8975228505 165780.231992565)', 'POINT(-40273.5194220084 165070.862992153)', 'POINT(-42793.8908064418 165807.262484494)', 'POINT(-41592.0753122211 165327.793695076)', 1420070708, 1420071057, datetime.time(6, 0), '503_0_U_45'), (7272, 'AANT5', 'CRJ2', 'POINT(-37772.6753019521 166280.950647465)', 'POINT(-40040.3571925886 163640.231674948)', 'POINT(-37792.1517335032 166100.028290137)', 'POINT(-37576.1084555906 165679.025513595)', 1